<a href="https://colab.research.google.com/github/SeoyoungOhMe/skt-fly-ai-challenger-class/blob/master/02_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Komoran
import numpy as np
komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."
# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

['오늘', '날씨', '구름']


In [ ]:
# 단어 사전 구축 및 단어별 인덱스 부여
dics = {}
for word in nouns:
  if word not in dics.keys():
    dics[word] = len(dics)
print(dics)

{'오늘': 0, '날씨': 1, '구름': 2}


In [ ]:
# 원-핫 인코딩
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
  with open(filename, 'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:] # header 제거
  return data

In [ ]:
# 측정 시작
start = time.time()
# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료:  0.5855550765991211


In [ ]:
# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)

2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  149.85265064239502


In [ ]:
# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1)
print('3) word2vec 모델 학습 완료: ', time.time() - start)

3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  662.184562921524


In [ ]:
# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  1192.3759033679962


In [ ]:
# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

corpus_count :  200000
corpus_total_words :  1076896


In [ ]:
from gensim.models import Word2Vec
# 모델 로딩
model = Word2Vec.load('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

corpus_total_words :  1076896


In [ ]:
# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])

사랑 :  [ 6.72609136e-02 -6.42251074e-01  2.49599218e-01  3.70102078e-01
 -1.36663973e-01  4.85151522e-02 -1.25460148e-01  3.74669544e-02
 -2.01850780e-03  4.53179777e-01 -4.09352481e-01 -1.74659804e-01
  1.73184946e-01 -2.27571741e-01 -1.82308644e-01  3.55393112e-01
  1.07869849e-01 -1.78833991e-01 -2.51225889e-01 -3.65720958e-01
  6.31793737e-01  1.99121684e-01 -1.92007562e-03  2.27498040e-01
 -2.39849150e-01  7.47290999e-02  1.71114311e-01  1.05624512e-01
 -8.45654756e-02 -3.77291590e-01  2.31273726e-01  2.34409407e-01
  1.52297601e-01 -4.28643711e-02 -3.36723737e-02 -4.24629040e-02
  1.60527810e-01  1.57465905e-01 -9.66719389e-02 -3.73948775e-02
 -3.16848814e-01  2.80476123e-01  1.20469593e-02 -6.65556252e-01
  4.42181170e-01  3.49654585e-01  3.98509030e-04 -9.13718250e-03
  3.41865897e-01 -7.13858679e-02  1.80992126e-01 -4.94560637e-02
 -2.91445047e-01  6.62863255e-02 -1.26952991e-01 -1.35764927e-01
 -1.08342290e-01 -3.45770836e-01 -7.55091310e-02 -1.09776646e-01
  2.60686785e-01  1

In [ ]:
# 단어 유사도 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))

일요일 = 월요일	 0.9258787
안성기 = 배우	 0.72757363
대기업 = 삼성	 0.876305
일요일 != 삼성	 0.64457506
히어로 != 삼성	 0.45462903


In [ ]:
# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))

[('설경구', 0.9457056522369385), ('정재영', 0.9419457316398621), ('고소영', 0.9377008676528931), ('한석규', 0.9366825222969055), ('박신양', 0.9366332292556763)]
[('엑스맨', 0.8101549744606018), ('포터', 0.8059613108634949), ('해리', 0.7920705080032349), ('반지의 제왕', 0.781561553478241), ('미이라', 0.778642475605011)]


In [ ]:
from konlpy.tag import Okt # 트위터 한국어 처리기 - 일상 생활에 등장하는 단어에 유용
from konlpy.tag import Komoran

In [ ]:
# 파일 형식 확인
import chardet
import os
file_name = './train_game.json'

with open(file_name, 'r') as f:
    result = chardet.detect(f.read())

print(result['encoding'])

TypeError: open() takes 1 positional argument but 2 were given

In [ ]:
import json
with open('./train_game.json', 'r') as f:
  json_file = json.load(f)

TypeError: open() takes 1 positional argument but 2 were given

In [ ]:
import json

def open_json(file_path):
    with open(file_path, 'r') as file:
      data = json.load(file)
    return data

In [ ]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

start = time.time()

print('1) 말뭉치 데이터 읽기 시작')
train = open_json("/content/train_game.json")
print(len(train))
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

1) 말뭉치 데이터 읽기 시작


TypeError: open() takes 1 positional argument but 2 were given

In [ ]:
train[0]

In [ ]:
# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = #Komoran()
docs = [komoran.nouns(sentence["sentence"]) for sentence in train]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)

In [ ]:
# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1) # skip-gram
print('3) word2vec 모델 학습 완료: ', time.time() - start)

In [ ]:
# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

In [ ]:
# 모델 로딩
model = Word2Vec.load('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

In [ ]:
# 가장 유사한 단어 추출
print(model.wv.most_similar("퀘스트", topn=5))
print(model.wv.most_similar("사냥", topn=5))